In [1]:
sc = spark.sparkContext

VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
1,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark.version

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'3.4.1-amzn-2'

In [3]:
from enum import Enum


class TaxiColors(str, Enum):
    GREEN = "green"
    YELLOW = "yellow"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import os

TAXI_DIR_PATH = "s3://robot-dreams-source-data/home-work-1/nyc_taxi/"
YELLOW_TAXI_DIR_PATH = os.path.join(TAXI_DIR_PATH, TaxiColors.YELLOW)
GREEN_TAXI_DIR_PATH = os.path.join(TAXI_DIR_PATH, TaxiColors.GREEN)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Use just for schema reading

df_yellow_limited = (spark.read.options(
    mergeSchema="false",
    recursiveFileLookup="true",
).parquet(YELLOW_TAXI_DIR_PATH).limit(1))

df_green_limited = (spark.read.options(
    mergeSchema="false",
    recursiveFileLookup="true",
).parquet(GREEN_TAXI_DIR_PATH).limit(1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Print schema of both dfs
from itertools import zip_longest


def get_sorted_schema(schema):
    return sorted(schema, key=lambda x: x.name)


for c_yellow, c_green in zip_longest(
        get_sorted_schema(df_yellow_limited.schema),
        get_sorted_schema(df_green_limited.schema)):
    print(f"Yellow: {c_yellow}\t\tGreen: {c_green}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Yellow: StructField('DOLocationID', LongType(), True)		Green: StructField('DOLocationID', LongType(), True)
Yellow: StructField('PULocationID', LongType(), True)		Green: StructField('PULocationID', LongType(), True)
Yellow: StructField('RatecodeID', LongType(), True)		Green: StructField('RatecodeID', LongType(), True)
Yellow: StructField('VendorID', LongType(), True)		Green: StructField('VendorID', LongType(), True)
Yellow: StructField('airport_fee', IntegerType(), True)		Green: StructField('congestion_surcharge', IntegerType(), True)
Yellow: StructField('congestion_surcharge', DoubleType(), True)		Green: StructField('ehail_fee', IntegerType(), True)
Yellow: StructField('extra', DoubleType(), True)		Green: StructField('extra', DoubleType(), True)
Yellow: StructField('fare_amount', DoubleType(), True)		Green: StructField('fare_amount', DoubleType(), True)
Yellow: StructField('improvement_surcharge', DoubleType(), True)		Green: StructField('improvement_surcharge', DoubleType(), True)
Yel

In [7]:
# Get differences in schema

for diff in set(df_green_limited.schema) ^ set(df_yellow_limited.schema):
    print(diff)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructField('lpep_dropoff_datetime', TimestampNTZType(), True)
StructField('tpep_dropoff_datetime', TimestampNTZType(), True)
StructField('lpep_pickup_datetime', TimestampNTZType(), True)
StructField('congestion_surcharge', DoubleType(), True)
StructField('ehail_fee', IntegerType(), True)
StructField('airport_fee', IntegerType(), True)
StructField('trip_type', DoubleType(), True)
StructField('tpep_pickup_datetime', TimestampNTZType(), True)
StructField('congestion_surcharge', IntegerType(), True)

In [9]:
# Creating schemas and read whole dataframes

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampNTZType, IntegerType
from pyspark.sql.functions import col

common_schema = StructType([
    StructField('VendorID', LongType(), True),

    #     StructField('tpep_pickup_datetime', TimestampNTZType(), True), # available only in yellow
    #     StructField('tpep_dropoff_datetime', TimestampNTZType(), True), # available only in yellow

    #     StructField('lpep_pickup_datetime', TimestampNTZType(), True), # available only in green
    #     StructField('tpep_dropoff_datetime', TimestampNTZType(), True), # available only in green

    #     StructField('passenger_count', DoubleType(), True), # available type in green
    #     StructField('passenger_count', LongType(), True), # available type in yellow

    #     StructField('RatecodeID', LongType(), True),  # available type in green
    #     StructField('RatecodeID', DoubleType(), True),  # available type in green
    
    #     StructField('payment_type', LongType(), True),  # changed because of error
    #     StructField('payment_type', DoubleType(), True),  # available type in green
    
    StructField('PULocationID', LongType(), True), # green
    StructField('trip_distance', DoubleType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('DOLocationID', LongType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('congestion_surcharge', DoubleType(), True),  # Use Double here
    StructField(
        'trip_type', DoubleType(), True
    )  # available only in green, but use it in both, yellow will be null-valued

    #     StructField('airport_fee', IntegerType(), True), # yellow
    #     StructField('ehail_fee', IntegerType(), True), # green
])

yellow_schema = StructType([
    StructField('RatecodeID', LongType(), True),
    StructField('payment_type', LongType(), True),
    StructField('passenger_count', LongType(), True),
    StructField('tpep_pickup_datetime', TimestampNTZType(), True),
    StructField('tpep_dropoff_datetime', TimestampNTZType(), True),
    StructField('airport_fee', IntegerType(), True), *common_schema
])

green_schema = StructType([
    StructField('RatecodeID', DoubleType(), True),
    StructField('payment_type', DoubleType(), True),
    StructField('passenger_count', DoubleType(), True),
    StructField('lpep_pickup_datetime', TimestampNTZType(), True),
    StructField('lpep_dropoff_datetime', TimestampNTZType(), True),
    StructField('ehail_fee', IntegerType(), True), *common_schema
])


df_yellow = (
    spark.read.options(
        mergeSchema="false",
        recursiveFileLookup="true",
    ).schema(yellow_schema)
    .parquet(
        *(os.path.join(TAXI_DIR_PATH, f"yellow/{year}")
        for year in range(2014, 2023) if year not in (2019, 2021)),
        os.path.join(TAXI_DIR_PATH, "yellow/2023/yellow_tripdata_2023-01.parquet")
    )
    .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
    .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
    .withColumnRenamed("airport_fee", "fee")
)

df_green = (
    spark.read.options(
        mergeSchema="false",
        recursiveFileLookup="true",
    ).schema(green_schema)
     .parquet(
        *(os.path.join(TAXI_DIR_PATH, f"green/{year}")
        for year in range(2014, 2023)),
        os.path.join(TAXI_DIR_PATH, "green/2023/green_tripdata_2023-01.parquet")
    )
    .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")
    .withColumnRenamed("lpep_dropoff_datetime","dropoff_datetime")
    .withColumnRenamed("ehail_fee", "fee")
    .withColumn("passenger_count",col("passenger_count").cast(LongType()))
    .withColumn("RatecodeID",col("RatecodeID").cast(LongType()))
    .withColumn("payment_type",col("payment_type").cast(LongType()))
    # Parsing these columns to Long, not Double cause this data doesnt need decimal places.    
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Code smell because of 2023 different schema

df_yellow_2023 = (
    spark.read.options(
        mergeSchema="false",
        recursiveFileLookup="true",
    ).schema(StructType([
         *(field 
          for field in yellow_schema
          if field.name not in ("PULocationID", "DOLocationID")
        ),
        StructField('PULocationID', IntegerType(), True),
        StructField('DOLocationID', IntegerType(), True),

    ]))
    .parquet(os.path.join(TAXI_DIR_PATH, "yellow/2023/yellow_tripdata_2023-0[2-9].parquet"))
    .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
    .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
    .withColumnRenamed("airport_fee", "fee")
    .withColumn("PULocationID", col("PULocationID").cast(LongType()))
    .withColumn("DOLocationID", col("PULocationID").cast(LongType()))
)

df_green_2023 = (
    spark.read.options(
        mergeSchema="false",
        recursiveFileLookup="true",
    ).schema(StructType([
        *(field 
          for field in green_schema
          if field.name not in ("PULocationID", "DOLocationID")
        ), 
        StructField("PULocationID", IntegerType(), True),
        StructField('DOLocationID', IntegerType(), True)
    ]))
    .parquet(os.path.join(TAXI_DIR_PATH, "green/2023/green_tripdata_2023-0[2-9].parquet"))
    .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")
    .withColumnRenamed("lpep_dropoff_datetime","dropoff_datetime")
    .withColumnRenamed("ehail_fee", "fee")
    .withColumn("passenger_count", col("passenger_count").cast(LongType()))
    .withColumn("RatecodeID", col("RatecodeID").cast(LongType()))
    .withColumn("payment_type", col("payment_type").cast(LongType()))
    .withColumn("PULocationID", col("PULocationID").cast(LongType()))
    .withColumn("DOLocationID", col("PULocationID").cast(LongType()))
    # Parsing these columns to Long, not Double cause this data doesnt need decimal places.    
)

df_yellow = df_yellow.unionByName(df_yellow_2023)
df_green = df_green.unionByName(df_green_2023)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.sql.functions import lit


df_green = df_green.withColumn("taxi_type", lit(TaxiColors.GREEN))
df_yellow = df_yellow.withColumn("taxi_type", lit(TaxiColors.YELLOW))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_yellow.select("passenger_count", "RatecodeID", "payment_type", "taxi_type").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+------------+---------+
|passenger_count|RatecodeID|payment_type|taxi_type|
+---------------+----------+------------+---------+
|              6|         1|           1|   yellow|
|              5|         1|           1|   yellow|
|              5|         1|           1|   yellow|
|              5|         1|           2|   yellow|
|              2|         1|           2|   yellow|
+---------------+----------+------------+---------+
only showing top 5 rows

In [13]:
# Use unioning by name, I dont sure col positions are equal

raw_trips_df = df_yellow.unionByName(df_green)
print(df_green.count() + df_yellow.count() == raw_trips_df.count(), raw_trips_df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True 827697864

In [14]:
raw_trips_df.select("pickup_datetime", "dropoff_datetime").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|    pickup_datetime|   dropoff_datetime|
+-------------------+-------------------+
|2014-01-01 00:02:00|2014-01-01 00:04:00|
|2014-01-01 00:06:00|2014-01-01 00:09:00|
|2014-01-01 00:10:00|2014-01-01 00:13:00|
|2014-01-01 00:54:00|2014-01-01 00:55:00|
|2014-01-01 00:29:18|2014-01-01 00:35:13|
+-------------------+-------------------+
only showing top 5 rows

In [15]:
# Вилучити поїздки з відстанню < 0.1 км, тарифом < $2, тривалістю < 1 хв.

from pyspark.sql.functions import unix_timestamp, dayofweek, date_trunc, hour
from pyspark.sql.functions import round as sp_round

raw_trips_df_filtered = raw_trips_df.withColumns({
    "pickup_hour": hour("pickup_datetime"),
    "pickup_day_of_week": dayofweek(col("pickup_datetime")),
    "duration_min": sp_round(
        (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) / 60,
        4
    ) # long casting doesnt work with not-tz
}).filter(
    (col("trip_distance") >= 0.1) &
    (col("total_amount") >= 2) &
    (col("duration_min") >= 1 )
)

raw_trips_df_filtered.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

816187143

In [16]:
TAXI_ZONE_LOOKUP_URL = os.path.join(TAXI_DIR_PATH, "taxi_zone_lookup.csv")

taxi_zone_df = spark.read.csv(TAXI_ZONE_LOOKUP_URL, header=True)
taxi_zone_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows

In [17]:
# Виконати JOIN з taxi_zone_lookup.csv, додавши поля pickup_zone, dropoff_zone.
from pyspark.sql.functions import broadcast

trips_df_joined = raw_trips_df_filtered.join(
        broadcast(
            taxi_zone_df.select(
                col("Zone").alias("pickup_zone"),
                col("LocationID").alias("PULocationID"),
            )
        ),
        how="left",
        on="PULocationID"
    ).join(
        broadcast(
            taxi_zone_df.select(
                col("Zone").alias("dropoff_zone"),
                col("LocationID").alias("DOLocationID"),
            )
        ),
        how="left",
        on="DOLocationID" 
    )

trips_df_joined.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+----------+------------+---------------+-------------------+-------------------+----+--------+-------------+------------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+---------+---------+-----------+------------------+------------+--------------------+--------------+
|DOLocationID|PULocationID|RatecodeID|payment_type|passenger_count|    pickup_datetime|   dropoff_datetime| fee|VendorID|trip_distance|store_and_fwd_flag|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|trip_type|taxi_type|pickup_hour|pickup_day_of_week|duration_min|         pickup_zone|  dropoff_zone|
+------------+------------+----------+------------+---------------+-------------------+-------------------+----+--------+-------------+------------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+---------+--------

In [27]:
from pyspark.sql.functions import count, avg
from pyspark.sql.functions import max as sp_max, min as sp_min, sum as sp_sum

zone_summary = (
    trips_df_joined.groupBy("pickup_zone").agg(
        count("*").alias("total_trips"),
        avg("trip_distance").alias("avg_trip_distance"),
        avg("total_amount").alias("avg_total_amount"),
        avg("tip_amount").alias("avg_tip_amount"),
        (avg((col("taxi_type") == "yellow").cast("int")) * 100).alias("yellow_share"),
        (avg((col("taxi_type") == "green").cast("int")) * 100).alias("green_share"),
        sp_max("trip_distance").alias("max_trip_distance"),
        sp_min("tip_amount").alias("min_tip_amount")
    )
)

zone_summary.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------+
|         pickup_zone|total_trips| avg_trip_distance|  avg_total_amount|    avg_tip_amount|      yellow_share|       green_share|max_trip_distance|min_tip_amount|
+--------------------+-----------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------+
|         Westerleigh|       1254| 8.929792663476873| 39.18330143540669|2.5147448165869215|  48.5645933014354| 51.43540669856459|            124.4|           0.0|
|Charleston/Totten...|       1747|28.510601030337728| 89.57432741843154|2.3209616485403552| 87.06353749284487|12.936462507155122|            103.0|           0.0|
|      Pelham Parkway|      97611|16.476772802245677|18.982729405497654|0.5824950056858343|10.123858991302209| 89.87614100869779|        221944.04|           0.0|
|           Rego Park|

In [33]:
from datetime import date

RESULT_URL = "s3://lavreniuk-emr-storage/results/"
ZONE_URL = os.path.join(RESULT_URL, f"zone_statistic/{date.today()}/zone_statistic.parquet")
zone_summary.coalesce(1).write.mode("overwrite").parquet(ZONE_URL) # doint coalesce to get 1 file
# We can`t directly pass a name of .parquet files, pyspark does it by itself

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
from pyspark.sql.functions import when, col

zone_days_statstic_df = (
    trips_df_joined.groupBy("pickup_day_of_week", "pickup_zone").agg(
        count("*").alias("total_trips"),
        (avg((col("fare_amount") > 30).cast("int")) * 100).alias("high_fare_share"),
    ).withColumn(
        "pickup_day_of_week", 
        (when(col("pickup_day_of_week") == 1, "Sunday")
            .when(col("pickup_day_of_week") == 2, "Monday")
            .when(col("pickup_day_of_week") == 3, "Tuesday")
            .when(col("pickup_day_of_week") == 4, "Wednesday")
            .when(col("pickup_day_of_week") == 5, "Thursday")
            .when(col("pickup_day_of_week") == 6, "Friday")
            .when(col("pickup_day_of_week") == 7, "Saturday")
        )
    )
)

zone_days_statstic_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+-----------+------------------+
|pickup_day_of_week|        pickup_zone|total_trips|   high_fare_share|
+------------------+-------------------+-----------+------------------+
|         Wednesday|  Crotona Park East|       8260|  10.0363196125908|
|           Tuesday|  Crotona Park East|       7819|11.395319094513365|
|            Sunday|         Bath Beach|       1871|31.694281133083912|
|          Thursday|Morningside Heights|    1087313| 4.044925426257205|
|          Thursday|         Highbridge|      23351| 10.87747848057899|
+------------------+-------------------+-----------+------------------+
only showing top 5 rows

In [34]:
ZONE_DAYS_URL = os.path.join(RESULT_URL, f"zone_days_statstic/{date.today()}/zone_days_statstic.parquet")

zone_days_statstic_df.coalesce(1).write.mode("overwrite").parquet(ZONE_DAYS_URL)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
test_df_zone = spark.read.options(
    mergeSchema="false",
    recursiveFileLookup="true",
).parquet(ZONE_URL).limit(5)

test_df_zone.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------+
|         pickup_zone|total_trips| avg_trip_distance|  avg_total_amount|    avg_tip_amount|      yellow_share|       green_share|max_trip_distance|min_tip_amount|
+--------------------+-----------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------+
|         Westerleigh|       1254| 8.929792663476873| 39.18330143540669| 2.514744816586922|  48.5645933014354| 51.43540669856459|            124.4|           0.0|
|Charleston/Totten...|       1747| 28.51060103033772| 89.57432741843157|2.3209616485403552| 87.06353749284487|12.936462507155122|            103.0|           0.0|
|      Pelham Parkway|      97611|16.476772802245684| 18.98272940549765|0.5824950056858342|10.123858991302209| 89.87614100869779|        221944.04|           0.0|
|           Rego Park|

In [36]:
test_df_zone_days = spark.read.options(
    mergeSchema="false",
    recursiveFileLookup="true",
).parquet(ZONE_DAYS_URL).limit(5)

test_df_zone_days.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+-----------+------------------+
|pickup_day_of_week|        pickup_zone|total_trips|   high_fare_share|
+------------------+-------------------+-----------+------------------+
|         Wednesday|  Crotona Park East|       8260|  10.0363196125908|
|           Tuesday|  Crotona Park East|       7819|11.395319094513365|
|            Sunday|         Bath Beach|       1871|31.694281133083912|
|          Thursday|Morningside Heights|    1087313| 4.044925426257205|
|          Thursday|         Highbridge|      23351| 10.87747848057899|
+------------------+-------------------+-----------+------------------+